# Layoffs data cleaning

Import necessary library to load the data

In [106]:
import pandas as pd

In [107]:
df = pd.read_csv(r"layoffs.csv")

## See the First few records of data

In [108]:
df.head()

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
0,Atlassian,Sydney,Other,500.0,0.05,3/6/2023,Post-IPO,Australia,210.0
1,SiriusXM,New York City,Media,475.0,0.08,3/6/2023,Post-IPO,United States,525.0
2,Alerzo,Ibadan,Retail,400.0,NaN,3/6/2023,Series B,Nigeria,16.0
3,UpGrad,Mumbai,Education,120.0,NaN,3/6/2023,Unknown,India,631.0
4,Loft,Sao Paulo,Real Estate,340.0,0.15,3/3/2023,Unknown,Brazil,788.0


## Check the detail on the dataframe

In [109]:
df.shape[0]

2361

In [110]:
df.describe()

,total_laid_off,percentage_laid_off,funds_raised_millions
count,1621.000000,1576.000000,2152.000000
mean,238.358421,0.257917,822.454134
std,769.718801,0.255575,5593.961626
min,3.000000,0.000000,0.000000
25%,36.000000,0.100000,50.000000
50%,80.000000,0.170000,156.500000
75%,180.000000,0.300000,445.000000
max,12000.000000,1.000000,121900.000000


In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2361 entries, 0 to 2360
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   company                2361 non-null   object 
 1   location               2361 non-null   object 
 2   industry               2357 non-null   object 
 3   total_laid_off         1621 non-null   float64
 4   percentage_laid_off    1576 non-null   float64
 5   date                   2360 non-null   object 
 6   stage                  2355 non-null   object 
 7   country                2361 non-null   object 
 8   funds_raised_millions  2152 non-null   float64
dtypes: float64(3), object(6)
memory usage: 166.1+ KB


# Perform data cleaning

## 1. Check for duplicates and copy the data into a dataframe with removed duplicates

In [112]:
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
2356    False
2357     True
2358     True
2359     True
2360     True
Length: 2361, dtype: bool

In [113]:
df.duplicated().sum()

5

In [114]:
df[df.duplicated(subset=["company", "location", "industry", "total_laid_off", "percentage_laid_off", "date", "stage", "country", "funds_raised_millions"])]

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
1492,Cazoo,London,Transportation,750.0,0.15,6/7/2022,Post-IPO,United Kingdom,2000.0
2357,Yahoo,SF Bay Area,Consumer,1600.0,0.20,2/9/2023,Acquired,United States,6.0
2358,Hibob,Tel Aviv,HR,70.0,0.30,3/30/2020,Series A,Israel,45.0
2359,Casper,New York City,Retail,NaN,NaN,9/14/2021,Post-IPO,United States,339.0
2360,Wildlife Studios,Sao Paulo,Consumer,300.0,0.20,11/28/2022,Unknown,Brazil,260.0


In [115]:
df_layoffv2 = df[
    ~df.duplicated(
        subset=[
            "company", "location", "industry", "total_laid_off",
            "percentage_laid_off", "date", "stage", "country",
            "funds_raised_millions"
        ],
        keep=False
    )
]

## 2. Perform Standardization - check columns

### Trim the company

In [116]:
df_layoffv2["company"] = df_layoffv2["company"].str.strip()

C:\Users\Clifford\AppData\Local\Temp\ipykernel_51448\901894105.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_layoffv2["company"] = df_layoffv2["company"].str.strip()


### Check all the distinct industry

In [117]:
sorted(df_layoffv2["industry"].dropna().unique())

['Aerospace',
 'Construction',
 'Consumer',
 'Crypto',
 'Crypto Currency',
 'CryptoCurrency',
 'Data',
 'Education',
 'Energy',
 'Fin-Tech',
 'Finance',
 'Fitness',
 'Food',
 'HR',
 'Hardware',
 'Healthcare',
 'Infrastructure',
 'Legal',
 'Logistics',
 'Manufacturing',
 'Marketing',
 'Media',
 'Other',
 'Product',
 'Real Estate',
 'Recruiting',
 'Retail',
 'Sales',
 'Security',
 'Support',
 'Transportation',
 'Travel']

In [118]:
df_layoffv2["industry"].unique()

array(['Other', 'Media', 'Retail', 'Education', 'Real Estate',
       'Transportation', 'Marketing', nan, 'Healthcare', 'Security',
       'Food', 'Fitness', 'Consumer', 'Logistics', 'HR', 'Support',
       'Travel', 'Crypto', 'Finance', 'Data', 'Sales', 'Infrastructure',
       'Hardware', 'Product', 'Construction', 'Legal', 'Energy',
       'Manufacturing', 'Recruiting', 'Aerospace', 'Crypto Currency',
       'Fin-Tech', 'CryptoCurrency'], dtype=object)

'Crypto', 'CryptoCurrency' and 'Crypto Currency' are the same industry, check which one is the one to adapt and standardize

In [119]:
(df_layoffv2["industry"] == "CryptoCurrency").sum()

1

In [120]:
(df_layoffv2["industry"] == "Crypto Currency").sum()

2

In [121]:
(df_layoffv2["industry"] == "Crypto").sum()

99

In [122]:
df_layoffv2.loc[df_layoffv2["industry"] == "CryptoCurrency", "industry"] = "Crypto"
df_layoffv2.loc[df_layoffv2["industry"] == "Crypto Currency", "industry"] = "Crypto" 

In [123]:
(df_layoffv2["industry"] == "Crypto").sum()

102

### Check all distinct Country

In [124]:
sorted(df_layoffv2["country"].dropna().unique())

['Argentina',
 'Australia',
 'Austria',
 'Bahrain',
 'Belgium',
 'Brazil',
 'Bulgaria',
 'Canada',
 'Chile',
 'China',
 'Colombia',
 'Czech Republic',
 'Denmark',
 'Egypt',
 'Estonia',
 'Finland',
 'France',
 'Germany',
 'Ghana',
 'Greece',
 'Hong Kong',
 'Hungary',
 'India',
 'Indonesia',
 'Ireland',
 'Israel',
 'Italy',
 'Japan',
 'Kenya',
 'Lithuania',
 'Luxembourg',
 'Malaysia',
 'Mexico',
 'Myanmar',
 'Netherlands',
 'New Zealand',
 'Nigeria',
 'Norway',
 'Pakistan',
 'Peru',
 'Poland',
 'Portugal',
 'Romania',
 'Russia',
 'Senegal',
 'Seychelles',
 'Singapore',
 'South Africa',
 'South Korea',
 'Spain',
 'Sweden',
 'Switzerland',
 'Thailand',
 'Turkey',
 'United Arab Emirates',
 'United Kingdom',
 'United States',
 'United States.',
 'Uruguay',
 'Vietnam']

In [125]:
df_layoffv2["country"].unique()

array(['Australia', 'United States', 'Nigeria', 'India', 'Brazil',
       'United States.', 'France', 'Germany', 'Israel', 'Sweden',
       'United Kingdom', 'Japan', 'South Korea', 'China', 'Italy',
       'Singapore', 'Indonesia', 'Estonia', 'Canada', 'Ireland',
       'Finland', 'Netherlands', 'Spain', 'Portugal', 'Chile', 'Colombia',
       'Argentina', 'Seychelles', 'Austria', 'Mexico', 'Switzerland',
       'Egypt', 'Kenya', 'Luxembourg', 'Greece', 'Poland', 'Norway',
       'Belgium', 'Denmark', 'Hong Kong', 'New Zealand', 'Malaysia',
       'Hungary', 'Vietnam', 'Thailand', 'Lithuania', 'Ghana', 'Senegal',
       'Pakistan', 'United Arab Emirates', 'Peru', 'Bahrain', 'Romania',
       'Turkey', 'Russia', 'Uruguay', 'Bulgaria', 'South Africa',
       'Czech Republic', 'Myanmar'], dtype=object)

Standardize 'United States.' to 'United States'

In [126]:
df_layoffv2.loc[df_layoffv2["country"] == "United States.", "country"] = "United States"

In [127]:
df_layoffv2["country"].unique()

array(['Australia', 'United States', 'Nigeria', 'India', 'Brazil',
       'France', 'Germany', 'Israel', 'Sweden', 'United Kingdom', 'Japan',
       'South Korea', 'China', 'Italy', 'Singapore', 'Indonesia',
       'Estonia', 'Canada', 'Ireland', 'Finland', 'Netherlands', 'Spain',
       'Portugal', 'Chile', 'Colombia', 'Argentina', 'Seychelles',
       'Austria', 'Mexico', 'Switzerland', 'Egypt', 'Kenya', 'Luxembourg',
       'Greece', 'Poland', 'Norway', 'Belgium', 'Denmark', 'Hong Kong',
       'New Zealand', 'Malaysia', 'Hungary', 'Vietnam', 'Thailand',
       'Lithuania', 'Ghana', 'Senegal', 'Pakistan',
       'United Arab Emirates', 'Peru', 'Bahrain', 'Romania', 'Turkey',
       'Russia', 'Uruguay', 'Bulgaria', 'South Africa', 'Czech Republic',
       'Myanmar'], dtype=object)

### Convert the 'date' column to an actual date

In [128]:
df_layoffv2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2351 entries, 0 to 2356
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   company                2351 non-null   object 
 1   location               2351 non-null   object 
 2   industry               2347 non-null   object 
 3   total_laid_off         1613 non-null   float64
 4   percentage_laid_off    1568 non-null   float64
 5   date                   2350 non-null   object 
 6   stage                  2345 non-null   object 
 7   country                2351 non-null   object 
 8   funds_raised_millions  2142 non-null   float64
dtypes: float64(3), object(6)
memory usage: 183.7+ KB


In [129]:
df_layoffv2["date"] = pd.to_datetime(df_layoffv2["date"])

C:\Users\Clifford\AppData\Local\Temp\ipykernel_51448\3993264423.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_layoffv2["date"] = pd.to_datetime(df_layoffv2["date"])


In [130]:
df_layoffv2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2351 entries, 0 to 2356
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   company                2351 non-null   object        
 1   location               2351 non-null   object        
 2   industry               2347 non-null   object        
 3   total_laid_off         1613 non-null   float64       
 4   percentage_laid_off    1568 non-null   float64       
 5   date                   2350 non-null   datetime64[ns]
 6   stage                  2345 non-null   object        
 7   country                2351 non-null   object        
 8   funds_raised_millions  2142 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 183.7+ KB


In [131]:
df_layoffv2["date"].head()

0   2023-03-06
1   2023-03-06
2   2023-03-06
3   2023-03-06
4   2023-03-03
Name: date, dtype: datetime64[ns]

## 3. Check for any NULLs (NaN) - Company, Industry, Country, Total Laid off

In [132]:
df_layoffv2["company"].isna().sum()

0

In [133]:
df_layoffv2["industry"].isna().sum()

4

In [134]:
df_layoffv2["country"].isna().sum()

0

In [135]:
df_layoffv2["total_laid_off"].isna().sum()

738

In [136]:
df_layoffv2["percentage_laid_off"].isna().sum()

783

Check which rows have a NaN for industry

In [137]:
df_layoffv2[df_layoffv2["industry"].isna()]

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
8,Airbnb,SF Bay Area,NaN,30.0,NaN,2023-03-03,Post-IPO,United States,6400.0
330,Bally's Interactive,Providence,NaN,NaN,0.15,2023-01-18,Post-IPO,United States,946.0
736,Juul,SF Bay Area,NaN,400.0,0.30,2022-11-10,Unknown,United States,1500.0
1595,Carvana,Phoenix,NaN,2500.0,0.12,2022-05-10,Post-IPO,United States,1600.0


### Proceed to updated industry = NaN, if the company, location were found

In [138]:
# 1. Create a helper dataframe containing the known industry values
lookup = df_layoffv2[df_layoffv2["industry"].notna()][
    ["company", "location", "industry"]
].drop_duplicates()

# 2. Merge (self‑join) to bring in the industry from matching rows
df_merged = df_layoffv2.merge(
    lookup,
    on=["company", "location"],
    how="left",
    suffixes=("", "_lookup")
)

# 3. Update NULL industry values using the lookup industry
df_merged["industry"] = df_merged["industry"].fillna(df_merged["industry_lookup"])

# 4. Drop the helper column
df_layoffv2 = df_merged.drop(columns=["industry_lookup"])

### Check if there is still a NaN industry

In [139]:
df_layoffv2[df_layoffv2["industry"].isna()]

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
333,Bally's Interactive,Providence,NaN,NaN,0.15,2023-01-18,Post-IPO,United States,946.0


### Check why it was not updated, check if the record is unique

In [140]:
df_layoffv2[df_layoffv2["company"].str.contains("Ball", regex=True, na=False)]

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
333,Bally's Interactive,Providence,NaN,NaN,0.15,2023-01-18,Post-IPO,United States,946.0


### Remove records of data that has "total_laid_off" and "percentage_laid_off" = NaN, this is not useful in data exploration

In [141]:
# df_layoffv2 = df_layoffv2[(df_layoffv2["total_laid_off"].isna() & df_layoffv2["percentage_laid_off"].isna())]
df_layoffv2 = df_layoffv2.dropna(subset=["total_laid_off", "percentage_laid_off"],how="all")

Get total records

In [142]:
df_layoffv2.shape[0]

2025

# Export the file as a new CSV layoffs_v2

In [143]:
df_layoffv2.to_csv("layoffs_v2.csv", index=False)